In [ ]:
#docker pull tensorflow/serving
#sudo docker run -p 8501:8501 --mount type=bind,source=/home/henrique/Documents/UFRJ/pedrenrique-analytics/models/classifier,target=/models/classifier -e MODEL_NAME=classifier -t tensorflow/serving &

In [1]:
import os
import numpy as np
from PIL import Image

In [2]:
def load_image(filename):
    image = Image.open(filename).convert("RGB").resize((150, 150))
#     display(image)
    return np.asarray(image)

In [3]:
images = []
labels = []

In [4]:
for subfolder in [x for x in os.listdir("datasets") if "ipynb" not in x]:
    for filename in [x for x in os.listdir(os.path.join("datasets", subfolder)) if "ipynb" not in x]:
        images.append(load_image(os.path.join("datasets", subfolder, filename)))
        labels.append(subfolder)

In [5]:
images = np.array(images)/255

In [6]:
idx = np.random.choice(np.arange(images.shape[0]), 5)
idx

array([114, 188, 179,  63,  26])

In [8]:
X = images[idx]
X.shape

(5, 150, 150, 3)

In [9]:
y = np.array(labels)[idx]
y

array(['covid', 'pneumonia', 'pneumonia', 'normal', 'normal'], dtype='<U9')

In [10]:
import requests
import json

In [19]:
data = json.dumps({"instances": X.tolist()})

In [20]:
headers = {"content-type": "application/json"}

In [21]:
labels_order = np.array(["covid", "normal", "pneumonia"])

In [22]:
json_response = requests.post(url="http://localhost:8501/v1/models/classifier:predict", data=data, headers=headers)
json_response

<Response [200]>

In [23]:
predictions = json.loads(json_response.text)['predictions']
predictions

[[0.997745693, 0.0014638697, 0.000790396938],
 [2.3300915e-08, 1.3168434e-06, 0.999998689],
 [1.64663834e-05, 0.000123696489, 0.99985981],
 [1.93983069e-05, 0.995089114, 0.00489147427],
 [0.000190743609, 0.998728216, 0.00108098774]]

In [24]:
y_api = labels_order[np.argmax(np.array(predictions), axis=1)]

In [25]:
print(y)
print(y_api)

['covid' 'pneumonia' 'pneumonia' 'normal' 'normal']
['covid' 'pneumonia' 'pneumonia' 'normal' 'normal']
